# Guided Project: Building Fast Queries on a CSV

In this project I will use a csv file adapted from the [Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices). The IDs have changed and the prices are made integers.


## Variable Descriptions:

**ID:** A unique identifier for the laptop.

**Company:** The name of the company that produces the laptop.

**Product:** The name of the laptop.

**TypeName:** The type of laptop.

**Inches:** The size of the screen in inches.

**ScreenResolution:** The resolution of the screen.

**CPU:** The laptop CPU.

**RAM:** The amount of RAM in the laptop.

**Memory:** The size of the hard drive.

**GPU:** The graphics card name.

**OpSys:** The name of the operating system.

**Weight:** The laptop weight.

**Price:** The price of the laptop.


## Goal of the project

The goal of this guided project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:

Given a laptop id, find the corresponding data.

Given an amount of money, find whether there are two laptops whose total price is that given amount.

Identify all laptops whose price falls within a given budget.

#### Exploring the data

In [1]:
import csv

In [6]:
with open('laptops.csv', encoding='utf8') as f:
    file = list(csv.reader(f))
    header = file[0]
    rows = file[1:]

In [7]:
header

['Id',
 'Company',
 'Product',
 'TypeName',
 'Inches',
 'ScreenResolution',
 'Cpu',
 'Ram',
 'Memory',
 'Gpu',
 'OpSys',
 'Weight',
 'Price']

In [8]:
rows[:5]

[['6571244',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 2.3GHz',
  '8GB',
  '128GB SSD',
  'Intel Iris Plus Graphics 640',
  'macOS',
  '1.37kg',
  '1339'],
 ['7287764',
  'Apple',
  'Macbook Air',
  'Ultrabook',
  '13.3',
  '1440x900',
  'Intel Core i5 1.8GHz',
  '8GB',
  '128GB Flash Storage',
  'Intel HD Graphics 6000',
  'macOS',
  '1.34kg',
  '898'],
 ['3362737',
  'HP',
  '250 G6',
  'Notebook',
  '15.6',
  'Full HD 1920x1080',
  'Intel Core i5 7200U 2.5GHz',
  '8GB',
  '256GB SSD',
  'Intel HD Graphics 620',
  'No OS',
  '1.86kg',
  '575'],
 ['9722156',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '15.4',
  'IPS Panel Retina Display 2880x1800',
  'Intel Core i7 2.7GHz',
  '16GB',
  '512GB SSD',
  'AMD Radeon Pro 455',
  'macOS',
  '1.83kg',
  '2537'],
 ['8550527',
  'Apple',
  'MacBook Pro',
  'Ultrabook',
  '13.3',
  'IPS Panel Retina Display 2560x1600',
  'Intel Core i5 3.1GHz',
  '8GB',
  '256GB SSD',
  'Intel

In [11]:
class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            file = list(csv.reader(f))
            self.header = file[0]
            self.rows = file[1:]
            for row in self.rows:
                row[-1] = int(row[-1])

In [12]:
instance = Inventory('laptops.csv')

In [13]:
instance.header

['Id',
 'Company',
 'Product',
 'TypeName',
 'Inches',
 'ScreenResolution',
 'Cpu',
 'Ram',
 'Memory',
 'Gpu',
 'OpSys',
 'Weight',
 'Price']

In [14]:
len(instance.rows)

1303

In [17]:
# Creating a function to extract the information of the laptop according to the id

class Inventory():
    def __init__(self, csv_filename):
        with open(csv_filename) as f:
            file = list(csv.reader(f))
            self.header = file[0]
            self.rows = file[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None


In [18]:
instance_test = Inventory('laptops.csv')


In [19]:
instance_test.get_laptop_from_id('3362737')

['3362737',
 'HP',
 '250 G6',
 'Notebook',
 '15.6',
 'Full HD 1920x1080',
 'Intel Core i5 7200U 2.5GHz',
 '8GB',
 '256GB SSD',
 'Intel HD Graphics 620',
 'No OS',
 '1.86kg',
 575]

In [21]:
print(instance_test.get_laptop_from_id('3362736'))

None


#### Notes:

The algorithm from the previous requires us to look at every single row to find the one that we are looking for (or decide that such a row does not exist). This algorithm has time complexity O(R) where R is the number of rows.

We've learned, in this mission, that we can solve this problem more efficiently by preprocessing the data. Using a set, we can check in constant time whether a given identifier exists. However, we don't just want to know if it exists, we also want to retrieve the remaining row information. Therefore, we will use a dictionary instead of a set. Dictionaries have the same fast lookup properties that sets have, but allow us to associate values to the keys.

In our dataset, we only have about 1,300 laptops, so it might seem unnecessary to improve the performance of this query. However, you have to imagine that this code could be used in situations where the inventory contains millions of rows. Also, if we perform a lot of queries, even on a small dataset, the slow query performance will start to add up. It might eventually become the bottleneck of the application.

The idea is proceprocess the data into a dictionary where the keys are the IDs and the values the rows. Then, we will use that dictionary in the get_laptop_from_id() method. We can do this by:

Preprocess the data and create the dictionary in the __init__() method.
Re-implement the get_laptop_from_id() method. We will do it as a new method so that we can compare the two.

In [22]:
# Adding efficiency to the class

class Inventory():
    def __init__(self, csv_filename):
        self.id_to_row = dict()
        with open(csv_filename) as f:
            file = list(csv.reader(f))
            self.header = file[0]
            self.rows = file[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                self.id_to_row[row[0]] = row
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row.keys():
            return self.id_to_row[laptop_id]
        else: 
            None


In [23]:
instance = Inventory('laptops.csv')

In [24]:
# Testing the new class
instance.get_laptop_from_id_fast('3362737')

['3362737',
 'HP',
 '250 G6',
 'Notebook',
 '15.6',
 'Full HD 1920x1080',
 'Intel Core i5 7200U 2.5GHz',
 '8GB',
 '256GB SSD',
 'Intel HD Graphics 620',
 'No OS',
 '1.86kg',
 575]

In [26]:
print(instance.get_laptop_from_id_fast('3362736'))

None


### Conclusions of last step:

The get_laptop_from_id() method has time complexity O(R) where R is the number of rows. In contrast, the new implementation as time complexity O(1). It does so by using more memory to store the self.id_to_row dictionary and using a bit more time creating an instance (because it needs to create the dictionary).


### Next Step:

Let's experiment to compare the performance of the two methods. The idea is to generate random IDs using the random module. Then, use both methods to lookup these same IDs. We will use the time module to measure the execution time of each lookup and, for each method, add all times together.


In [27]:
import time
import random

In [29]:
ids = [str(random.randint(1000000, 9999999)) for _ in range(10_000)]

In [30]:
# Testing with the previously created class
instance = Inventory("laptops.csv")

In [31]:
# Creating variable to aggregate the times of calling get_laptop_from_id()
total_time_no_dict = 0


In [32]:
for id in ids:
    start = time.time()
    instance.get_laptop_from_id(id)
    end = time.time()
    total_time_no_dict += (end-start)

In [33]:
total_time_dict = 0
for id in ids:
    start = time.time()
    instance.get_laptop_from_id_fast(id)
    end = time.time()
    total_time_dict += (end-start)

In [35]:
# Printing the total time using both functions
total_time_dict, total_time_no_dict

(0.0060214996337890625, 0.9624409675598145)

# Exercise 2 Situation:

Sometimes, your store offers a promotion where you give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore.

For example, imagine that there are only three laptops in inventory.
The prices of these laptops are 1,339, 898, and 575. Say we offered a gift card of 2,500. Since a customer can buy, at most, two laptops with a gift card, the maximum they can spend is 2,237 (1,339 plus 898). Therefore, they might feel cheated because, no matter how they spend their gift card, they cannot spend the full 2,500.

You don't want to make a customer feel cheated, so whenever you issue a gift card, you want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for D dollars, you want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

Write a function that, given a dollar amount, checks whether it is possible to spend precisely that amount by purchasing up to two laptops.

In [39]:
# Adding a method check_promotion_dollars() that takes two arguments: 
# self and dollars.

class Inventory():
    def __init__(self, csv_filename):
        self.id_to_row = dict()
        with open(csv_filename) as f:
            file = list(csv.reader(f))
            self.header = file[0]
            self.rows = file[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                self.id_to_row[row[0]] = row
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row.keys():
            return self.id_to_row[laptop_id]
        else: 
            None
            
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        
        for row in self.rows:
            for row_2 in self.rows:
                if(row_2[-1] + row[-1])==dollars:
                    return True
            return False
    
                        

In [40]:
# Testing the last addition
instance = Inventory('laptops.csv')

In [41]:
instance.check_promotion_dollars(1000)

True

In [43]:
instance.check_promotion_dollars(442)

False

### Improving the last method:

In the previous mission, we've learned how we can preprocess data to answer the kind of queries that we used in the check_promotion_dollars(). Let's implement this to make our code run faster.

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.

To check if there is a pair of laptops can be done in the same way as we've learned in the last mission.

In [61]:
# Adding a method check_promotion_dollars_fast() that takes two arguments: 
# self and dollars.

class Inventory():
    def __init__(self, csv_filename):
        self.id_to_row = dict()
        self.prices = set()
        with open(csv_filename) as f:
            file = list(csv.reader(f))
            self.header = file[0]
            self.rows = file[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                self.id_to_row[row[0]] = row
                self.prices.add(row[-1])
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row.keys():
            return self.id_to_row[laptop_id]
        else: 
            None
            
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        
        for row in self.rows:
            for row_2 in self.rows:
                if(row_2[-1] + row[-1])==dollars:
                    return True
        return False
        
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            for price2 in self.prices:
                if(price2 + price) == dollars:
                    return True
        return False

In [62]:
# Testing the latest addition
instance = Inventory("laptops.csv")

In [46]:
instance.check_promotion_dollars_fast(1000)

True

In [47]:
instance.check_promotion_dollars_fast(442)

False

#### Comparing the performance of the last two methods

In [48]:
prices = [random.randint(100, 5000) for _ in range(100)]

In [63]:
# creating a variable that will aggregate the times of calling
# check_promotion_dollars()
total_time_no_set = 0
for price in prices:
    start = time.time()
    instance.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += (end-start)

In [64]:
total_time_set = 0
for price in prices:
    start = time.time()
    instance.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += (end-start)

In [65]:
print(total_time_no_set, total_time_set)

2.4783623218536377 0.43169093132019043


## Using Binary Search

I am going to leverage and extend the algorithm to help a customer find all laptops that fall within their budget.

More formally, I want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price it at most D.

If I sort all laptops by price, I can use binary search to identify the first laptop in the sorted list with a price larger than D. I need to make sure that my binary search finds the first one on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than D dollars

In [80]:
class Inventory():
    def __init__(self, csv_filename):
        self.id_to_row = dict()
        self.prices = set()
        with open(csv_filename) as f:
            file = list(csv.reader(f))
            self.header = file[0]
            self.rows = file[1:]
            for row in self.rows:
                row[-1] = int(row[-1])
                self.id_to_row[row[0]] = row
                self.prices.add(row[-1])
        
        def row_price(row):
            return row[-1]
        self.row_by_price = sorted(self.rows, key=row_price)
    
    
                
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row.keys():
            return self.id_to_row[laptop_id]
        else: 
            None
            
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True
        
        for row in self.rows:
            for row_2 in self.rows:
                if(row_2[-1] + row[-1])==dollars:
                    return True
        return False
        
    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price in self.prices:
            for price2 in self.prices:
                if(price2 + price) == dollars:
                    return True
        return False
    
    # Adding method for binary search
    def find_first_laptop_more_expensive(self, price):
        range_start = 0
        range_end = len(self.row_by_price) - 1
        
        while range_start < range_end:
            range_middle = (range_start + range_end)//2
            new_price = self.row_by_price[range_middle][-1]
            
                            
            
            
            elif new_price >= price:
                range_end = range_middle
            else:
                range_start = range_middle+1
                
        if self.row_by_price[range_start][-1] > price:
            return range_start
        else:
            return -1
        

In [81]:
# testing
instance = Inventory('laptops.csv')

In [82]:
print(instance.find_first_laptop_more_expensive(1000))

683


In [83]:
print(instance.find_first_laptop_more_expensive(10000))

-1


## Pushing the project further

If you want to push this project further, we suggest that you think about the following queries:

Imagine that we extend our budget query to take as input a range of prices, min_price and max_price, rather than a single price. Write a query that finds all laptops whose price is in the given range.
Sometimes, a customer wants a laptop with some characteristics such as, for instance, 8GB or RAM and a 256GB hard drive. It would be interesting for those customers to provide a way to find the cheapest laptop that matches the desired characteristics. For simplicity, focus only on the amount of RAM and hard drive capacity. You might need to convert those values to integers rather than using strings.